# **Convert Original Dataset to Llama2 Model required data format for fine tune**

**Note - This script executed in Google Colab**

- Here Conversational original dataset **timdettmers/openassistant-guanaco** to Human Q in between **[INST] and [/INST]** and **system answer without any token**. This is the format we needed to Fine tune the **Llama2 LLM model.**
- Each LLM model's need its fine tune dataet in different way, for Llam2 this is the way. So always we need to check the Models document to understand more

- Original data: **{'text': ['### Human: What is Mango? ### Assistant: Mango is a Fruit]}**
- Converted EX: **[INST] What is Mango? [/INST] Mango is a Fruit**
- Here **What is Mango?** is **Q** and **Mango is a Fruit** is **ans** provided by model which will be used as fine tune dataset

- Once we reformat the data, we will upload this to **Hugging face dataset** location - **PrabhaB/guanaco-llama2-1k**
  - transformed_dataset.push_to_hub("PrabhaB/guanaco-llama2-1k")

## **Original data**
- Original data: https://huggingface.co/datasets/timdettmers/openassistant-guanaco?row=0


## **Original data in different format, that we need to reformat**
- Reformat Data 1K sample: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k?row=2

- Complete Reformat Data: https://huggingface.co/datasets/mlabonne/guanaco-llama2

## **How to Prepare reformat data**
- How to Prepare reformat data: https://colab.research.google.com/drive/1Ad7a9zMmkxuXTOh1Z7-rNSICA4dybpM2?usp=sharing
- 4a_Guanaco_Llama_Dataset_Reformat_push_to_hf_hub.ipynb

## **How to push data to hub**
- How to push data to hub: https://huggingface.co/docs/datasets/v1.16.0/upload_dataset.html#:~:text=Upload%20your%20files,lines%2C%20text%2C%20and%20Parquet.

## **Install Hugging face and its dataset**

In [1]:
!pip install -q datasets
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tok

## **Login to Hugging face with Read access to Read Original Datset**

In [9]:
from datasets import load_dataset
import re

# Load the dataset from Huggging Face
dataset = load_dataset('timdettmers/openassistant-guanaco')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [11]:
dataset[:2] #Here each Q &A starts with 'text' and each Q starts with '### Human' and ans with '### Assistant:'

{'text': ['### Human: Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo?### Assistant: Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país.',
  '### Human: Самый великий человек из всех живших на планете?### Assistant: Для начала нужно выбрать критерии величия челов

## **Function to Reformat original dataset and read 1k records**

In [4]:
# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text'] #Here each Q &A starts with 'text' and
    segments = conversation_text.split('###') #Each Q starts with '### Human' and ans with '### Assistant:', So split that as new

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip() # Replace word Human='' and Pass Human Q to human_text

        # Check if there is a corresponding assistant(Ans for each Q) segment before processing,
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip() # Replace word Assistant ='' and Pass Assistant Ans to assistant_text


            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>') # Convert to reformat form
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}



/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


## **Apply Reformat Function**

In [12]:
# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

## **Final Reformat and orginal data**

In [13]:
#Original data
dataset[0:1]

{'text': ['### Human: Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo?### Assistant: Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país.']}

In [7]:
#Transformed data
transformed_dataset[0:1]

{'text': ['<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>']}

> **Above it converted Conversational original dataset to Human Q in between [INST] and [/INST] and system answer without any token**. This is the format we needed to Fine tune the Llama2 LLM model.

> Add blockquote


- Each LLM model's need its fine tune dataet in different way, for Llam2 this is the way. So always we need to check the Models document to understand more

- EX: [INST] What is Mango? [/INST] Mango is a Fruit
- Here What is Mango? is Q and Mango is a Fruit is ans

## **Before that Follow below steps in Hugging face to create Datset Repo**
- Create a **repository in Hugging face** under dataset. Here we created repo with name **guanaco-llama2-1k**
  - Follw this: https://huggingface.co/docs/datasets/v1.16.0/upload_dataset.html#:~:text=Upload%20your%20files,lines%2C%20text%2C%20and%20Parquet
- Upload your files manualy from local as mentioned in the above link or follow below python way
- **Local Way**
  - Once you have created a repository, navigate to the Files and versions tab to add a file. Select Add file to upload your dataset files.
  - **Create a Dataset card:** The Dataset card is essential for helping users find your dataset and understand how to use it responsibly.

## **Login to Hugging face Write access to write to Datset**

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [15]:
transformed_dataset.push_to_hub("PrabhaB/guanaco-llama2-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/PrabhaB/guanaco-llama2-1k/commit/e365fe94b51cb1dc6cc109c1cad9fa33103a3529', commit_message='Upload dataset', commit_description='', oid='e365fe94b51cb1dc6cc109c1cad9fa33103a3529', pr_url=None, pr_revision=None, pr_num=None)

> Now loaded dataset to Dataset

## **END**